# Tara's Capstone Milestone Report

### An introduction to the problem

The client, SumOfUs, is a non-profit advocacy group that organizers consumers, workers and shareholders to speak out against the growing power of large corporations in our society.  To accomplish this, SumOfUs maintains a subscription list of members that have signed up to receive action alerts via email.  These email alerts link to online petitions, fundraising appeals, and other suggested actions to impact issues related to corporate power.  At the time of this writing, the SumOfUs email subscription list was just over 4 million.  Over 12.3 million people have taken an advocacy action through SumOfUs since its inception in 2012.

The primary growth engine of the SumOfUs membership list has always been peer-to-peer social sharing of content via email and social media.  When a user takes an action on our site (ie, signs a petition), they are typically directed to a post-action page that asks the member to share the action on Facebook, Twitter, or via a personal email. These shares reach new users, who come to our page and take action, thus becoming members themselves.

SumOfUs is a multi-issue advocacy organization that runs campaigns on a wide-ranging variety topics: climate change, human rights, trade agreements, habitat destruction, and discrimination, to name just a few.  When new members join through social sharing on a particular campaign, we consider these members to be part of a “campaign cohort”, and we monitor the aggregate performance of that cohort.  We have noticed that there is a great deal of variance between these campaign cohorts in regards to subsequent engagement and donation rates.  Some campaigns attract new subscribers that will consistently respond to action alerts to sign petitions and make donations at high rates.  Other campaigns attract members that will rarely act or donate after joining.

As the organization’s lead data analyst, I am frequently asked these question by campaigners: “How "good" are the new members of this campaign’s cohort? Did our campaign attract members that will contribute to our organization's impact and sustainability in the long-term, or are those members likely to never return to our site again? Should we spend our finite resources to further cultivate this cohort and attempt to replicate its success, or should we move on to more promising campaigns?”

Campaigners often ask these questions very soon after a campaign has been launched..  Until now, I haven’t had a strong analytical tool or technique to predict member quality from the early indicators I can calculate.  My instinct is that cohort quality was influenced by a number of factors, such as the regional distribution of the members, the specific topic of the campaign, the size of the cohort, and the ratio of current members to new members that sign a particular petition.  I do have access to some early activity indicators, such as engagement and donation rates in the early days of the cohort, but did not have a formula to take all of these factors into account. 

Furthermore, we believe that one of the most predictive factors to campaign quality is the issue area, or topic, of the campaign.  Campaigners usually assign each campaign a topic 'tag' - a small bit of text that explains the campaigns topic category.  However, these topic tags are not properly standardized, and cannot be used on their own to represent useful topics for modelling.

Therefore, my project has two steps:
* Discover and assign the topics of each campaign using Latent Dirichlet Allocation (LDA)
* Build a regression model to predict the donor propensity for each campaign cohort, using the LDA topics as features along with other indicators of campaign virality, age, regional distribution, and early cohort activity.

### What important fields and information does the data set have?

The main dataset of historic campaigns has the following features/ response:
    * unique campaign id
    * date campaign was created
    * count of how many joined on each campaign
    * count of how many existing members acted on the main mailing
    * % joining from SoU's main regions (USA, Canada, UK, Australia, New Zealand, Non-UK Europe, Rest of World)
    * number of opens, per member, in the first 7 days
    * number of clicks, per member, in the first 7 days
    * number of actions, per member, in the first 7 days  
    * number of donations, per member, in the first 7 days 
    * amount of donations, per member, in the first 7 days 
    * proportion of the cohort who donated in their first 90 days after joining
    
The secondary dataset is related to the topic analysis and contains:
    * unique campaign id
    * topic tag
    * full html of the campaign mailing

### What are its limitations i.e. what are some questions that you cannot answer with this data set?

We suspect that mobile usage proportions may be a key feature for predicting donations, with mobile users less likely to join.  However, the data does not contain relible mobile usage data for the period, so it was omitted.

The data does not contain any information about whether users shared campaigns on social media, which we believe to be a predicitive factor.  However, SoU does not have reliable data about social media sharing at this time.

The LDA algorithm used to determine topics may not discover the exact topics that the organization considers to be its main issue areas, due to differences in semantic expressions.  For instance, campaigns related to trade agreements will almost always contain words like 'trade', 'agreement', 'negotiate', 'secret', which are easy for the algorithm to cluster.  On the other hand, animal rights campaigns might not have enough common words to cluster, since one campaign might use words like 'lion','trophy','endangered','hunting' and another might use 'bunny','angora','fur','cruelty'.  While the organization thinks of these campaigns as both being under the 'animal rights' issue category, and would prefer to make predictions based on an animal rights category, there are enough common words for the LDA to identify them as a cluster.

### What kind of cleaning and wrangling did you need to do?

Producing the datasets required extensive wrangling in SQL.

For the LDA dataset, I needed to find a list of english language petition campaigns within my date range. Each petition campaign has several mailings that link to it.  These mailings are all very similar but usually not identical; they may be A/B testing variations or mailings sent to slightly different audiences.  In order to have a clean dataset for the LDA, I needed to return only one mailing for each campaign.  I used a nth value window function on my Redshift SQL server to find the 'top mailing', ie. the mailing responsible for the most actions on the campaign.

However, my Redshift server has a vendor-imposed limit of 256 characters per text field.  In this case, I needed to extract the full html of each mailing, far beyond that character limit.  All the data on my Redshift server is synced from a MySQL server, from a different vendor.  The MySQL server has many drawbacks compared to Redshift: I only have read-only access, it is far too slow to run complicated queries, and MySQL does not supporrt window functions, making operations like nth value very laborious.  However, I can use MySQL to access the complete html of each mailing.

I ended up generating the list of mailings using Redshift, and then exporting it as a CSV.  From the MySQL server, I exported all of the mailing HTML.  I then imported both files into a MySQL MAMP server and joined them.  I exported the joined table and imported it into Pandas for the LDA.

For the main dataset, I went through a lot of iterations of the queries to get the exact right set.  I needed campaigns that were Engligh language petitions in the right date range, and had over 500 new joiners within 3 days of the top mailing being sent out. After selecting the campaigns, I then needed to aggregates the regions and activity histories of the new joiners.  I exported that to Pandas, where I joined with the LDA table.  Finally, I normalized all the feature rows prior to running the predictive model.

### Are there other datasets you can find, use and combine with, to answer the questions that matter?

One feature I thought about including was the unsubscribe proportion - what percentage of new joiners unsubscribe within the first 7 days after joining.  This may be predictive, but it is difficult to calculate given our schema, and I feel that we already have enough features to model.  However, I may revist this.

### Based on these findings, what approach are you going to take? How has your approach changed from what you initially proposed, if applicable?

Originally, I had assumed that the campaigners would want the cohort predictions during the testing phase of the campaign.  Before any action alert is mass-mailed to the membership, drafts are sent to small list segments, in order to judge member interest and to A/B test messages, tone, subject lines, images, formatting, etc.  I wanted to know if I could predict the eventual cohort quality based on the early returns from the tests.  That way, the Campaigns team could make their decision on how to move forward with each campaign before the mass mailing even went out, informed by the modelled predictions.

However, there were serious problems with that approach.  Each campaign has a unique testing plan, with various numbers of tests, to various sized audiences, spread widely in time.  Furthermore, virality can be driven by the news cycle, and early test groups are often either more or less saturated with news than the bulk of the eventual cohort.  Sharing activity can be affected by the number of times a user has previously seen a petition shared, and this too would create a variance between cohorts. Modelling at the testing phase introduced too many variables, some that I could not control for.

At this point, I requested a meeting with the Growth team leader, to make sure I fully understood the business case.   The team leader clarified that it was not necessary to make a prediction before the large mass mailing.   Instead, he would be satisfied to have cohort quality projections two weeks after the mass mailing had been sent.  He explained that they were confident in their current testing program to determine which mailings to send to the full membership.  However, they were less satisfied with the information they needed to determine a follow-up strategy for the cohort, and needed more business intelligence regarding very similar campaigns that they might run in the coming weeks or months.

The two of us worked together to formulate a new modelling plan.   Instead of looking at early cohort joiners, I will only include members who joined a cohort within 72 hours of the mass-mailing being sent. 